In [1]:
import numpy as np
from numpy.linalg import inv
import sys
import os
import pandas as pd
import numpy as np
# sys.path.append('/root/barcode/')
sys.path.append('../')

from BarcodeScanner import tree_and_clustering, base_barcode
from itertools import product, combinations
from sklearn.linear_model import LinearRegression
import timeit

In [2]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
# from interaction_permutation_importance import *
import matplotlib.pyplot as plt

def gen_X(num_var: int, sample_size : int):
    data_dictionary = {}
    for i in range(num_var):
        var_name = "x" + f"{i + 1}"
        data_dictionary[var_name] = list(np.random.binomial(1, .5, sample_size))
    return pd.DataFrame(data_dictionary)

sys.path.append('/root/barcode/')
from BarcodeScanner import tree_and_clustering, base_barcode
from importance_score import f_test_result
import scipy
from scipy.stats import f
from sklearn.base import BaseEstimator, clone
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics      import r2_score

import numpy as np; import pandas as pd
from typing import List, Union

# itertools
from itertools import combinations

from utils import *


class one_way_ANOVA(BaseEstimator):
    def __init__(self):
        pass
    
    def fit(self, df, response_var_name =None):
        if response_var_name:
            try:
                df[response_var_name]
                self.response_var_name = response_var_name
            except:
                self.response_var_name = df.columns.tolist()[-1]
        else:
            self.response_var_name = df.columns.tolist()[-1]
        self.independet_var_name = df.columns[~df.columns.str.contains(self.response_var_name)].tolist()
        self.segment_means = df.groupby(self.independet_var_name)[self.response_var_name].mean().reset_index()
        self.segment_means.columns= self.segment_means.columns.tolist()[:-1] + ['y_hat']
        output = df.merge(self.segment_means, how = 'left', on = self.independet_var_name)
        y = output[self.response_var_name].to_numpy().reshape(-1)
        y_pred = output['y_hat'].to_numpy().reshape(-1)
        self._rsq = r2_score(y, y_pred)
        self.sse  = np.square(y - y_pred).sum()
        self.MSE_ = self.sse/(y.shape[0])
        output['sq'] = (output.y_hat - output[self.response_var_name])**2
        summary = output.groupby(self.independet_var_name).agg({"y_hat": np.mean, "sq": [lambda x: x.sum()/x.count(), 'count']}).reset_index()
        summary.columns = self.independet_var_name + ['group_means','mse','count']
        self._summary_table = summary
        return self
    

    @property
    def score(self):
        if hasattr(self, '_rsq'):
            return self._rsq
        else:
            raise AttributeError("fit the estimator first")
    
    @property
    def MSE(self):
        if hasattr(self, "MSE_"):
            return self.MSE_
        else:
            raise AttributeError("fit the estimator first")
class anova_clustering(base_barcode):
    def __init__(self, X, y):
        super().__init__(X, y)
        self.clean_data()
        self.estimator = one_way_ANOVA()
        self.fit()

    def clean_data(self):
        full_df = self.X.copy()
        full_df['y'] = self.y.reshape(-1)
        sorted_index = full_df.sort_values(full_df.columns.tolist()[:-1]).index.tolist()
        self.full_df = full_df.loc[sorted_index, :].reset_index(drop = True)
        self.barcode_df = pd.DataFrame(zip(self.barcode.reshape(-1), self.y.reshape(-1)), columns = ['z','y']).loc[sorted_index, :].reset_index(drop = True)
        self.X = full_df.iloc[:, :-1]
        self.y = full_df.y.to_numpy().reshape(-1,1);
        self.num_full_var = 2**self.X.shape[1]
        self.num_main_var = self.X.shape[1]
        del sorted_index
        del full_df
        
    def fit(self):   
        self.estimator.fit(self.full_df)
        self._summary_table = self.estimator._summary_table
        self._fit = True

    def gen_clustering_summary_table(self, num_clusters):
        original_summary = self.summary_table.copy()
        original_summary = original_summary.drop(['group_means','mse', 'count'], axis = 1)
        original_summary['y_pred'] = self.get_projected_mu_hat(num_clusters)
        full_df = self.full_df.copy()
        full_df = full_df.merge(original_summary, on = self.estimator.independet_var_name, how = 'left')
        return full_df
    
    def metric_after_clustering(self, num_clusters, metric = r2_score):
        full_df = self.gen_clustering_summary_table(num_clusters)
        return metric(full_df.y, full_df.y_pred)



    @property
    def summary_table(self):
        if hasattr(self, "_summary_table"):
            pass
        else:
            self._summary_table = self.predict_from_training()
        return self._summary_table
    
    @property
    def init_mu_hat(self):
        if hasattr(self, "_init_mu_hat"):
            pass
        else:
            self._init_mu_hat = self.summary_table.group_means.tolist()
            self._init_mu_hat = np.array(self._init_mu_hat).reshape(-1)
        return self._init_mu_hat
    
    @property
    def init_mu_var(self):
        if hasattr(self, "_init_mu_var"):
            pass
        else:
            self._init_mu_var = np.diag(self.estimator.MSE/self.summary_table['count'])
        return self._init_mu_var
    
    @property
    def init_beta_hat(self):
        if hasattr(self, "_init_beta_hat"):
            pass
        else:
            self._init_beta_hat = self.L_inv @ self.init_mu_hat
        return self._init_beta_hat

    @property
    def init_beta_var(self):
        if hasattr(self, "_init_beta_var"):
            pass
        else:
            self._init_beta_var = self.L_inv @ self.init_mu_var @ self.L_inv.T
        return self._init_beta_var

    @property
    def clustering_init_kwargs(self):
        summary = self.summary_table
        kwargs = {"means": summary.group_means.tolist(), "variances": summary.mse.tolist(), "sample_sizes": summary['count'].tolist()}
        return kwargs
    
    @property
    def init_cluster_idx(self):
        if hasattr(self, "_init_cluster_idx"):
            pass
        else:
            sample_sizes = self.clustering_init_kwargs['sample_sizes']
            clusters = []
            last = 0
            for n in sample_sizes:
                clusters.append([x for x in range(last, n+last)])
                last += n
            self._init_cluster_idx = clusters
        return self._init_cluster_idx

    @property
    def init_pdist(self):
        if hasattr(self, "_init_pdist"):
            pass
        else:
            self._init_pdist = self.pairwise_distances_from_means_variances(**self.clustering_init_kwargs)
        return self._init_pdist

    def cluster(self, n_clusters, save = True):
        self.last_n_clusters = n_clusters
        pdist = self.init_pdist.copy()
        init_cluster_idx = self.init_cluster_idx.copy()
        result = self.agglomerative_clustering(pairwise_distances = pdist, n_clusters = n_clusters, clusters = init_cluster_idx)
        cluster_idx = result[0]
        final_pdist = result[1]
        cluster_df = {}
        for cluster_id, cluster_index in enumerate(cluster_idx):
            cluster_id_name = f"cluster_{cluster_id}"
            cluster_df[cluster_id_name] = self.full_df.loc[cluster_index,:].copy()
            cluster_df[cluster_id_name] = cluster_df[cluster_id_name].groupby(self.original_columns).agg(np.mean).reset_index()
            cluster_df[cluster_id_name]['barcode'] = self.gen_barcode((cluster_df[cluster_id_name].loc[:, self.original_columns]))
        if save:
            self._latest_cluster_dfs = cluster_df
        else:
            self._latest_cluster_dfs = None
        
        return {"cluster": cluster_df, "final_pdist": final_pdist}

    def gen_mu_contrast_from_cluster(self, n_clusters, use_latest_cluster_df = True):
        if use_latest_cluster_df:
            if hasattr(self, '_latest_cluster_dfs'):
                if (self._latest_cluster_dfs != None) & (self.last_n_clusters == n_clusters):
                    cluster_df_list = self._latest_cluster_dfs
        try:
            cluster_df_list
        except:
            cluster_result = self.cluster(n_clusters = n_clusters)
            cluster_df_list = cluster_result['cluster']
            
        contrast_matrix = 0
        for cluster_name, cluster_df in cluster_df_list.items():
            if cluster_df.shape[0] > 1:
                contrast = np.zeros((cluster_df.shape[0]-1, 2**len(self.original_columns)))
                for i, row in enumerate(contrast):
                    row[cluster_df.barcode[0]] = 1
                    row[cluster_df.barcode[i+1]] = -1
                    contrast[i] = row
                if isinstance(contrast_matrix, np.ndarray):
                    contrast_matrix = np.concatenate([contrast_matrix, contrast], axis = 0)
                else:
                    contrast_matrix = contrast
            else:
                pass
        return contrast_matrix
    
    def gen_projection_matrix_mu(self, n_clusters):
        C = self.gen_mu_contrast_from_cluster(n_clusters)
        projection_matrix = C.T @ np.linalg.inv(C @ C.T) @ C
        projection_matrix = np.identity(C.shape[1]) - projection_matrix
        return projection_matrix
    
    def get_projected_mu_hat(self, n_clusters):
        mu = self.init_mu_hat
        proj_mu = self.gen_projection_matrix_mu(n_clusters = n_clusters) @ mu
        return proj_mu
    
    def get_projected_mu_hat_var(self, n_clusters):
        mu_var = self.init_mu_var
        proj = self.gen_projection_matrix_mu(n_clusters = n_clusters)
        proj_mu_var = proj @ mu_var @ proj.T
        return proj_mu_var
    
    def get_projected_beta_hat(self, n_clusters):
        proj_mu = self.get_projected_mu_hat(n_clusters = n_clusters)
        proj_beta = self.L_inv @ proj_mu
        return proj_beta
    
    def get_projected_beta_hat_var(self, n_clusters):
        mu_var = self.get_projected_mu_hat_var(n_clusters)
        return self.L_inv @ mu_var @ self.L_inv.T

    
    def ward_linkage(self, pairwise_distances, clusters, merge_indices):
        clusters = clusters.copy()
        i, j = merge_indices
        cluster_i = clusters[i]
        cluster_j = clusters[j]
        n_i = len(cluster_i)
        n_j = len(cluster_j)
        n = pairwise_distances.shape[1]
        new_distances = pairwise_distances.copy()
        new_distances = np.delete(new_distances, merge_indices, axis = 0)
        new_distances = np.delete(new_distances, merge_indices, axis = 1)
        new_distances = np.append(new_distances, np.zeros((1, new_distances.shape[1])), axis = 0)
        new_distances = np.append(new_distances,  np.zeros((new_distances.shape[0], 1)), axis = 1)
        clusters.append(cluster_i + cluster_j)
        clusters.remove(cluster_i)
        clusters.remove(cluster_j)
        col = 0
        
        for k in range(n):  # Subtract 2 because we've already added a row and column
            if k != i and k != j:
                n_k = len(clusters[col])
                n_all = n_i + n_j + n_k
                dist_ik = pairwise_distances[i, k]*(n_k + n_i)/n_all
                dist_jk = pairwise_distances[j, k]*(n_j + n_k)/n_all
                dist_ij = pairwise_distances[i, j]*(n_k)/n_all
                new_dist = dist_ik + dist_jk - dist_ij
                # Add other linkage methods here if needed
                
                new_distances[-1, col] = new_dist
                new_distances[col, -1] = new_dist
                col += 1

        return new_distances, clusters
    
    def gen_reduced_clustering(self, n_clusters):
        return self.agglomerative_clustering(pairwise_distances= self.init_pdist, n_clusters = n_clusters)

    def agglomerative_clustering(self, pairwise_distances, n_clusters, clusters = None, estimator = None):
        n = pairwise_distances.shape[0]
        if clusters:
            pass
        else:
            clusters = [[i] for i in range(n)]
        
        assert len(clusters) == n
        
        for k in range(n - n_clusters):
            min_dist = np.inf
            merge_indices = None
            
            for i in range(len(clusters)):
                for j in range(i + 1, len(clusters)):
                    dist = pairwise_distances[i, j]
                    if dist < min_dist:
                        min_dist = dist
                        merge_indices = (i, j)
            
            if merge_indices:
                pairwise_distances, clusters = self.ward_linkage(pairwise_distances, clusters, merge_indices)
        
        return clusters, pairwise_distances
    
    def pairwise_distances_from_means_variances(self, means, variances, sample_sizes):
        num_clusters = len(means)
        pairwise_distances = np.zeros((num_clusters, num_clusters))
        
        for i in range(num_clusters):
            for j in range(i + 1, num_clusters):
                n_i = sample_sizes[i]
                n_j = sample_sizes[j]
                mean_i = means[i]
                mean_j = means[j]
                var_i = variances[i]
                var_j = variances[j]
                
                # Calculate the pairwise distance using Ward linkage formula
                numerator = (n_i * n_j / (n_i + n_j)) * (mean_i - mean_j)**2
                denominator = np.sqrt((n_i * var_i + n_j * var_j) / (n_i + n_j))
                
                pairwise_distances[i, j] = np.sqrt(numerator / denominator)
                pairwise_distances[j, i] = pairwise_distances[i, j]
        
        return pairwise_distances

    def contrast_generator(self, max_variables = None):
        from itertools import product
        from scipy.sparse import vstack
        all_interaction_betas = scipy.sparse.eye(self.num_full_var - self.num_main_var -1, self.num_full_var, k = self.num_main_var + 1)
        betas_in_test = product([False, True], repeat = self.num_full_var - self.num_main_var -1)
        
        def get_new_contrasts(beta):
            vstacks = []
            for i, b in enumerate(beta):
                if b:
                    vstacks.append(all_interaction_betas.getrow(i))
            return vstack(vstacks).astype(np.uint8)
        if max_variables is not None:
            for beta in betas_in_test:
                if sum(beta) and (sum(beta) <= max_variables):
                    yield get_new_contrasts(beta)
        else:
            for beta in betas_in_test:
                if sum(beta):
                    yield get_new_contrasts(beta)


    @staticmethod    
    def partial_f_test(contrast, L_inv, cell_means, cell_means_covariance, sample_size):
        assert contrast.shape[0] < contrast.shape[1]
        assert contrast.shape[1] == L_inv.shape[0]
        assert L_inv.shape[0] == L_inv.shape[1]
        assert cell_means_covariance.shape == L_inv.shape
        from numpy.linalg import inv
        nu_1 = contrast.shape[0]
        nu_2 = sample_size - L_inv.shape[0]
        contrast = contrast.toarray()

        LC = contrast @ L_inv.T
        mu = LC @ cell_means
        var = LC @ cell_means_covariance @ LC.T
        try:
            f_value = mu.T @ inv(var) @ mu
            f_value /= nu_1
        except:
            f_value = 0
        
        return f_test_result(nu_1, nu_2, f_value)
    
    def gen_importance_score(self, num_clusters, filename = 'importance_score.csv', dir = os.getcwd(), max_variables = None):
        cell_means = self.get_projected_mu_hat(num_clusters)
        covariance = self.get_projected_mu_hat_var(num_clusters)
        kwargs = {"L_inv": self.L_inv, "cell_means": cell_means, "cell_means_covariance": covariance, "sample_size": self.X.shape[0]}

        from csv import writer
        from tqdm import tqdm
        with open(os.path.join(dir, filename), 'w') as f:
            csv_writer = writer(f)
            csv_writer.writerow(self.beta_names + ['score'])
            contrasts =  self.contrast_generator(max_variables = max_variables)
            from math import comb
            if max_variables:
                total_iter = sum([comb(self.num_full_var - self.num_main_var -1, x) for x in range(1, max_variables +1)])
            else:
                total_iter = sum([comb(self.num_full_var - self.num_main_var -1, x) for x in range(1,self.num_full_var - self.num_main_var)])
            for contrast in tqdm(contrasts, total = total_iter):
                result = self.partial_f_test(contrast = contrast, **kwargs)
                score = result.cdf
                row = contrast.sum(axis = 0).tolist()[0] + [score]
                csv_writer.writerow(row)
            else:
                from joblib import Parallel, delayed
                num_cores = os.cpu_count()
                batch_size = 1000 * num_cores
                partition_size = 1000
                contrast_batch = []
                for i, contrast in tqdm(enumerate(contrasts), total = total_iter):
                    contrast_batch.append(contrast)
                    if len(contrast_batch) % batch_size == 0:
                        f_test_results = Parallel(n_jobs=num_cores)(delayed(self.partial_f_test)(c, **kwargs) for c in contrast_batch)
                        f_test_results = [x.cdf for x in f_test_results]
                        rows = [c.sum(axis = 0).tolist()[0] + [f] for c,f in zip(contrast_batch, f_test_results)]
                        csv_writer.writerows(rows)
                        del contrast_batch
                        del rows
                        del f_test_results
                        contrast_batch = []
                # process the remainder
                if len(contrast_batch):
                    f_test_results = Parallel(n_jobs=num_cores)(delayed(self.partial_f_test)(c, **kwargs) for c in contrast_batch)
                    f_test_results = [x.cdf for x in f_test_results]
                    rows = [c.sum(axis = 0).tolist()[0] + [f] for c,f in zip(contrast_batch, f_test_results)]
                    csv_writer.writerows(rows)

from glob import glob
import pandas as pd
from dask import dataframe as dd
import numpy as np
import matplotlib.pyplot as plt
import joblib
@dataclass
class f_test_result:
    nu_1:int
    nu_2:int
    f_statistic: float
    
    @property
    def p_value(self):
        if hasattr(self, '_p_value'):
            pass
        else:
            sdf = f.sf(self.f_statistic, self.nu_1, self.nu_2, loc=0, scale=1)
            self._p_value = sdf
        return self._p_value
    
    @property
    def cdf(self):
        if hasattr(self, '_cdf'):
            pass
        else:
            self._cdf = 1 - self.p_value
        return self._cdf
from math import sqrt


In [3]:
x1 = [0,0,0,0,0,0,1,1,1,1]
x2 = [0,0,0,0,1,1,0,0,1,1]
y = [-3.655697,-4.848736,-5.537694,-8.661518,-8.486865,-11.669568,-3.312871,-2.031480,-8.475096,-5.116814]

In [4]:
df = pd.DataFrame(zip(x1, x2, y), columns = ['x1','x2','y'])

In [5]:
df

,x1,x2,y
0,0,0,-3.655697
1,0,0,-4.848736
2,0,0,-5.537694
3,0,0,-8.661518
4,0,1,-8.486865
5,0,1,-11.669568
6,1,0,-3.312871
7,1,0,-2.031480
8,1,1,-8.475096
9,1,1,-5.116814


In [6]:
barcode = anova_clustering(X = df.iloc[:, :-1], y = df.iloc[:, -1:])

In [7]:
barcode.init_mu_hat

array([ -5.67591125, -10.0782165 ,  -2.6721755 ,  -6.795955  ])

In [8]:
barcode.init_beta_hat

array([-5.67591125,  3.00373575, -4.40230525,  0.27852575])

In [9]:
barcode.estimator.MSE

2.5223245820265747

In [10]:
barcode.init_mu_var

array([[0.63058115, 0.        , 0.        , 0.        ],
       [0.        , 1.26116229, 0.        , 0.        ],
       [0.        , 0.        , 1.26116229, 0.        ],
       [0.        , 0.        , 0.        , 1.26116229]])

In [11]:
barcode.init_beta_var

array([[ 0.63058115, -0.63058115, -0.63058115,  0.63058115],
       [-0.63058115,  1.89174344,  0.63058115, -1.89174344],
       [-0.63058115,  0.63058115,  1.89174344, -1.89174344],
       [ 0.63058115, -1.89174344, -1.89174344,  4.41406802]])

In [12]:
inv(barcode.init_beta_var)

array([[3.96459681, 1.58583873, 1.58583873, 0.79291936],
       [1.58583873, 1.58583873, 0.79291936, 0.79291936],
       [1.58583873, 0.79291936, 1.58583873, 0.79291936],
       [0.79291936, 0.79291936, 0.79291936, 0.79291936]])

In [20]:
X = df.iloc[:, :-1]
X['1'] = 1
X['x1x2'] = X.x1*X.x2

In [24]:
X = X[['1','x1','x2','x1x2']].to_numpy()

In [33]:
(barcode.estimator.MSE * inv(X.T @ X)).round(4) == barcode.init_beta_var.round(4)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [34]:
C_beta = np.array([[0, 0, 1, 0],
                   [0, 0, 1, -1]])

In [35]:
C_mu = C_beta.dot(barcode.L.T)

In [36]:
C_mu

array([[0, 1, 0, 1],
       [0, 1, 0, 0]])

In [37]:
C_mu.dot(barcode.init_mu_hat.reshape(-1,1))

array([[-16.8741715],
       [-10.0782165]])

In [39]:
K  = C_beta @ barcode.L_inv

In [40]:
C_beta = np.array([[0, 0, 1, 0],
                   [0, 0, 1, -1]])

In [41]:
C_mu = C_beta @ barcode.L.T

In [58]:
barcode.L_inv @ barcode.init_mu_var @ barcode.L_inv.T

array([[ 0.63058115, -0.63058115, -0.63058115,  0.63058115],
       [-0.63058115,  1.89174344,  0.63058115, -1.89174344],
       [-0.63058115,  0.63058115,  1.89174344, -1.89174344],
       [ 0.63058115, -1.89174344, -1.89174344,  4.41406802]])

In [59]:
barcode.L @ barcode.L_inv @ barcode.init_mu_var @ barcode.L_inv.T @ barcode.L.T

array([[0.63058115, 0.        , 0.        , 0.        ],
       [0.        , 1.26116229, 0.        , 0.        ],
       [0.        , 0.        , 1.26116229, 0.        ],
       [0.        , 0.        , 0.        , 1.26116229]])

In [60]:
barcode.init_mu_var

array([[0.63058115, 0.        , 0.        , 0.        ],
       [0.        , 1.26116229, 0.        , 0.        ],
       [0.        , 0.        , 1.26116229, 0.        ],
       [0.        , 0.        , 0.        , 1.26116229]])

In [64]:
C_mu @ barcode.L_inv.T @ barcode.L_inv

array([[-1,  1,  0,  0],
       [-2,  2,  1, -1]])

In [61]:
K

array([[-1,  1,  0,  0],
       [-2,  2,  1, -1]])

In [43]:
C_mu @ barcode.L_inv.T

array([[ 0,  0,  1,  0],
       [ 0,  0,  1, -1]])

In [44]:
barcode.init_beta_var

array([[ 0.63058115, -0.63058115, -0.63058115,  0.63058115],
       [-0.63058115,  1.89174344,  0.63058115, -1.89174344],
       [-0.63058115,  0.63058115,  1.89174344, -1.89174344],
       [ 0.63058115, -1.89174344, -1.89174344,  4.41406802]])

In [45]:
barcode.L_inv @ barcode.init_mu_var @ barcode.L_inv.T

array([[ 0.63058115, -0.63058115, -0.63058115,  0.63058115],
       [-0.63058115,  1.89174344,  0.63058115, -1.89174344],
       [-0.63058115,  0.63058115,  1.89174344, -1.89174344],
       [ 0.63058115, -1.89174344, -1.89174344,  4.41406802]])

In [54]:
inv(C_mu @ barcode.init_mu_var @ C_mu.T)

array([[ 0.79291936, -0.79291936],
       [-0.79291936,  1.58583873]])

In [55]:
barcode.init_mu_hat.reshape(1, -1) @ K.T 

array([[-4.40230525, -4.680831  ]])

In [56]:
barcode.init_beta_hat.reshape(1, -1) @ C_beta.T

array([[-4.40230525, -4.680831  ]])

In [46]:
barcode.init_mu_hat.reshape(1, -1) @ C_mu.T @ inv(C_mu @ barcode.init_mu_var @ C_mu.T) @ C_mu @ barcode.init_mu_hat.reshape(-1, 1)

array([[117.15815897]])

In [47]:
barcode.init_mu_hat.reshape(1, -1) @ K.T @ inv(K @ barcode.init_mu_var @ K.T) @ K @ barcode.init_mu_hat.reshape(-1, 1)

array([[16.98669012]])

In [48]:
barcode.init_beta_hat.reshape(1, -1) @ C_beta.T @ inv(C_beta @ barcode.init_beta_var @ C_beta.T) @ C_beta @ barcode.init_beta_hat.reshape(-1, 1)

array([[16.98669012]])

In [65]:
inv(C_beta @ barcode.init_beta_var @ C_beta.T) 

array([[ 2.11445163, -0.79291936],
       [-0.79291936,  0.39645968]])

In [66]:
K

array([[-1,  1,  0,  0],
       [-2,  2,  1, -1]])

In [50]:
barcode.init_beta_hat

array([-5.67591125,  3.00373575, -4.40230525,  0.27852575])

In [51]:
barcode.init_mu_hat.reshape(1, -1) @ C_mu.T

array([[-16.8741715, -10.0782165]])

In [52]:
barcode.init_mu_hat.reshape(1, -1) @ K.T 

array([[-4.40230525, -4.680831  ]])